In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/pima-indians-diabetes.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
x = df.iloc[:,:8]
y = df.iloc[:,8]

Define a function that returns keras model itself

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2

In [7]:
def build_model(lambda_param):
    model = Sequential()
    model.add(Dense(8, input_dim=8, activation='relu', kernel_regularizer=l2(lambda_param)))
    model.add(Dense(8, activation='relu', kernel_regularizer=l2(lambda_param)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [8]:
np.random.seed(1)

In [10]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [12]:
model = KerasClassifier(build_fn=build_model)

/tmp/ipykernel_24242/2775510547.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=build_model)


In [13]:
lambda_param = [0.01, 0.1, 0.5, 0.1]
epochs = [50, 75, 100]
batch_size = [8, 16]
grid_params = dict(lambda_param=lambda_param, epochs=epochs, batch_size=batch_size)
grid_search = GridSearchCV(estimator=model, param_grid=grid_params, cv=5)

In [14]:
results = grid_search.fit(x,y)

2023-03-28 19:24:47.105593: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-28 19:24:47.105656: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-28 19:24:47.105684: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (neeraj-HP-Laptop): /proc/driver/nvidia/version does not exist
2023-03-28 19:24:47.106111: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
77/77 [==============================] - 0s 1ms/step - loss: 1.4322 - accuracy: 0.6270
Epoch 2/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7966 - accuracy: 0.6498
Epoch 3/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7727 - accuracy: 0.6515
Epoch 4/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7613 - accuracy: 0.6547
Epoch 5/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7576 - accuracy: 0.6515
Epoch 6/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7582 - accuracy: 0.6515
Epoch 7/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7334 - accuracy: 0.6531
Epoch 8/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7556 - accuracy: 0.6531
Epoch 9/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7398 - accuracy: 0.6547
Epoch 10/50
77/77 [==============================] - 0s 1ms/step - loss: 0.7397 - accuracy: 0.6547
Epoch 11/50
77/77 [

In [15]:
results.best_params_

{'batch_size': 8, 'epochs': 100, 'lambda_param': 0.1}

In [16]:
accuracy_mean = results.cv_results_['mean_test_score']
accuracy_std = results.cv_results_['std_test_score']
parameters = results.cv_results_['params']
print(results.best_score_)
print(results.best_params_)

0.7071471095085144
{'batch_size': 8, 'epochs': 100, 'lambda_param': 0.1}


In [22]:
def build_model(rate, lambda_param):
    model = Sequential()
    model.add(Dense(8, input_dim=8, activation='relu', kernel_regularizer=l2(lambda_param)))
    model.add(Dropout(rate))
    model.add(Dense(8, activation='relu', kernel_regularizer=l2(lambda_param)))
    model.add(Dropout(rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

In [25]:
model2 = KerasClassifier(build_fn=build_model, verbose=0)

/tmp/ipykernel_24242/2444103537.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model2 = KerasClassifier(build_fn=build_model, verbose=0)


In [26]:
rate = [0.1,0.2,0.3]
lambda_param = [0.1, 0.5]
epochs = [100]
batch_size = [8]
grid_params2 = dict(rate=rate, lambda_param=lambda_param, epochs=epochs, batch_size=batch_size)
grid_search2 = GridSearchCV(estimator=model2, param_grid=grid_params, cv=5)

In [27]:
results = grid_search2.fit(x,y)

In [28]:
accuracy_mean = results.cv_results_['mean_test_score']
accuracy_std = results.cv_results_['std_test_score']
parameters = results.cv_results_['params']
print(results.best_score_)
print(results.best_params_)

0.6797555446624756
{'batch_size': 8, 'epochs': 100, 'lambda_param': 0.1, 'rate': 0.2}
